In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import numba
from numba import jit
from scipy.io import wavfile
from IPython.display import Audio
import scipy
import librosa

In [4]:
from google.colab import drive, files

In [ ]:
Nx = 101
Nt = 500000
L = 0.7
dx = L / (Nx - 1)
f = 880
c = 2 * L * f
dt = 5e-6
l = 5e-5
gamma = 5e-5

ya = np.linspace(0, 0.01, 70)
yb = np.linspace(0.01, 0, 31)
y0 = np.concatenate([ya, yb])

sol = np.zeros((Nt, Nx))
sol[0] = y0
sol[1] = y0


@numba.jit("f8[:,:](f8[:,:], i8, i8, f8, f8, f8, f8)", nopython=True, nogil=True)
def compute_d(d, times, length, dt, dx, l, gamma):
    for t in range(1, times - 1):
        for i in range(2, length - 2):
            outer_fact = (1 / (c ** 2 * dt ** 2) + gamma / (2 * dt)) ** (-1)
            p1 = 1 / dx ** 2 * (d[t][i - 1] - 2 * d[t][i] + d[t][i + 1])
            p2 = 1 / (c ** 2 * dt ** 2) * (d[t - 1][i] - 2 * d[t][i])
            p3 = gamma / (2 * dt) * d[t - 1][i]
            p4 = l ** 2 / dx ** 4 * (d[t][i + 2] - 4 * d[t][i + 1] + 6 * d[t][i] - 4 * d[t][i - 1] + d[t][i - 2])
            d[t + 1][i] = outer_fact * (p1 - p2 + p3 - p4)
    return d


sol = compute_d(sol, Nt, Nx, dt, dx, l, gamma)
len(sol[::10, :])


def get_integral_fast(n):
    sin_arr = np.sin(n * np.pi * np.linspace(0, 1, 101))
    return np.multiply(sol, sin_arr).sum(axis=1)


hms = [get_integral_fast(n) for n in range(10)]

all_harmonics = True
if all_harmonics:
    tot = sol.sum(axis=1)[::10]  # all harmonics
else:
    tot = sum(hms)[::10]  # only first 10 harmonics
tot = tot.astype(np.float32)

In [8]:
wavfile.write('nusound.wav',20000,tot)

In [ ]:
f, t, Sxx = librosa.
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()